In [ ]:
#!pip install qiskit_ibm_runtime
#!pip install seaborn
#!pip install pylatexenc
#!pip install qiskit_aer
#!pip install 'qiskit[visualization]'

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_gate_map
import seaborn as sns
from matplotlib import pyplot as plt

# Providers

In [ ]:
# do it only once
QiskitRuntimeService.save_account(channel="ibm_quantum", token=TOKEN_ID, overwrite=True, set_as_default=True)

In [ ]:
services = QiskitRuntimeService()

In [ ]:
print(services.backends())

In [ ]:
for backend in services.backends():
    try:
        print(f"name={backend.name}, nqbits={backend.num_qubits}, status={backend.status().status_msg}, pending_jobs={backend.status().pending_jobs}")
        print("\tcoupling map:", backend.coupling_map)
        instructions = set()
        for inst, _ in backend.instructions:
            if isinstance(inst.name,str) and inst.name not in backend.configuration().basis_gates:
                instructions.add(inst.name)
        print("\tsupports: ", backend.configuration().basis_gates, "+", instructions)
    except:
        pass

In [ ]:
qpu_backend = services.least_busy(operational=True, min_num_qubits=5)
print("qpu selected:", qpu_backend.name, "n_qubit:", qpu_backend.num_qubits)

In [ ]:
# display some information about the qpu
print(qpu_backend.name)
# some dynamic information - for instance qubit 0 property
print("qubit 0 property:", qpu_backend.qubit_properties(0))
# or properties of ecr gate between qubit 0 and 1
print("ecr gate 0-1 property:", qpu_backend.target["ecr"][(0, 1)])
# readout properties on qubit 0
print("readout error on qubit 0:", qpu_backend.target["measure"][(0,)])

In [ ]:
plot_gate_map(qpu_backend)

In [ ]:
colors_palette = sns.dark_palette("red", 20)
colors = colors_palette.as_hex()
#We will use this color palette to represent meas and ecr errors
colors_palette

In [ ]:
measure_errors = []
for i in range(qpu_backend.num_qubits):
    measure_errors.append(qpu_backend.target["measure"][(i,)].error)
ecr_errors = []
for e in qpu_backend.target["ecr"]:
    ecr_errors.append(qpu_backend.target["ecr"][e].error)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(measure_errors, ax=ax1)
ax1.set_title('measure errors')
sns.histplot(ecr_errors, ax=ax2)
ax2.set_title('ECR errors')
plt.show()

In [ ]:
min_meas_error, max_meas_error = min(measure_errors), max(measure_errors)
print("measurement error, min:", min_meas_error, "max:", max_meas_error)
min_ecr_error, max_ecr_error = min(ecr_errors), max(ecr_errors)
print("ecr error, min:", min_ecr_error, "max:", max_ecr_error)

In [ ]:
display_meas_errors, display_ecr_errors = None, None
#for each measurement error, associate a color from 
###ENTER CODE HERE
###END CODE

In [ ]:
plot_gate_map(qpu_backend, qubit_color=display_meas_error, line_color=display_ecr_errors)

In [ ]:
#Can we check coupling map for simulator?
###ENTER CODE HERE
###END CODE

# Simulation

In [ ]:
# Our first circuit !
from qiskit import QuantumCircuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

circuit.draw(initial_state=True)
circuit.draw(output='mpl', style="iqp") # style="clifford"

In [ ]:
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram

In [ ]:
simulator_backend=AerSimulator()
job = simulator_backend.run(circuit, shots=1024)

In [ ]:
print(job.status())

In [ ]:
result_sim = job.result()
print(result_sim.get_counts())

In [ ]:
# Build a simulator from the qpu_backend - with its imperfections
local_qpu_simulator = AerSimulator.from_backend(qpu_backend)

# We do need to transpile the circuit for the local_qpu_simulator - check what happen if we don't
from qiskit.compiler import transpile
transpiled_circuit = transpile(circuit, local_qpu_simulator)
result_sim_qpu = local_qpu_simulator.run(transpiled_circuit, shots=1024).result()
print(result_sim_qpu.get_counts())

In [ ]:
# Run the job on the selected backend
from qiskit_ibm_runtime import SamplerV2 as Sampler
sampler = Sampler(mode=qpu_backend)
job_qpu = sampler.run([transpiled_circuit], shots=1024)
# We will check the results later

In [ ]:
plot_histogram([result_sim.get_counts(), result_sim_qpu.get_counts()], legend=["ideal", "noisy"])

In [ ]:
## Calculate fidelity with precision linked to sampling noise

###ENTER CODE HERE
###END CODE

In [ ]:
## How does fidelity change with circuit depth?

###ENTER CODE HERE
###END CODE

In [ ]:
# let us check statevector now
circuit2 = QuantumCircuit(2)
circuit2.h(0)
circuit2.cx(0, 1)
circuit2.save_statevector()

In [ ]:
from qiskit.visualization import plot_state_qsphere
sv1 = simulator_backend.run(circuit2).result().get_statevector()
plot_state_qsphere(sv1)

In [ ]:
sv1.draw('latex', prefix='The\\ state\\ vector:')

In [ ]:
## Can we do the same on noisy simulator? comment...

###ENTER CODE HERE
###END CODE

# Conditional X gate

Using `c_if` - you can make a gate conditional in your circuit - syntax is:

<code>
circuit.x(qreg).c_if(creg, 1)
</code>

Modify the circuit above to add a conditional X gate on the second qubit, if the value of the register is 0, what is the new circuit?

In [ ]:
qc_ghz = QuantumCircuit(3)
###ENTER CODE HERE
###END CODE

# Transpilation, Compilation, Assembling

You can use the following to check the size of your circuit:
<code>
qc.width()
qc.count_ops()
qc.size()
qc.depth()
</code>

Build a GHZ state $|000\rangle+|111\rangle$ and check these different values...

In [ ]:
###ENTER CODE HERE
###END CODE

print(f"initial: width={qc_ghz.width()}, counts_ops={qc_ghz.count_ops()}, size={qc_ghz.size()}, depth={qc_ghz.depth()}")

In [ ]:
# do the same on the decomposed circuit
qc_basis = qc_ghz.decompose()
print(f"decompose: width={qc_basis.width()}, counts_ops={qc_basis.count_ops()}, size={qc_basis.size()}, depth={qc_basis.depth()}")
qc_basis.draw(output='mpl')

In [ ]:
# ok - let transpile now the circuit and checks what it becomes on physical hardware
transp_3 = transpile(qc_ghz, local_qpu_simulator, optimization_level=3)
transp_3.draw(output='mpl')
print(f"transpiled: width={qc_basis.width()}, counts_ops={qc_basis.count_ops()}, size={qc_basis.size()}, depth={qc_basis.depth()}")

In [ ]:
# is there difference with optimization level 1 and 2
###ENTER CODE HERE
###END CODE

In [ ]:
# How is the circuit transpiled on the actual layout?
from qiskit.visualization import plot_circuit_layout
plot_circuit_layout(transp_3, qpu_backend)

In [ ]:
# Let us check on Assembling of transp_3
from qiskit import qasm3
qasm_string = qasm3.dumps(transp_3, experimental=qasm3.ExperimentalFeatures.SWITCH_CASE_V1)
print(qasm_string)

In [ ]:
# How does conditional gates assemble

###ENTER CODE HERE
###END CODE

# Remote jobs...
To retrieve a job from IBMQ you can just use `job = service.job('JOB_ID')`... `job` is the actual job object you could run locally.

In [ ]:
# retrieve the latest job looking at ID in your IBM Quantum Platform

In [ ]:
job = services.job(JOB_ID)
job_result = job_qpu.result()

In [ ]:
# The sample does not return count but a list of the samples
# display of the raw table
print(job_result[0].data.meas.array)
# display of the counts
print(job_result[0].data.meas.get_counts())

In [ ]:
## compare with fidelity of simulator